# HuggingFace 커스텀 프로젝트 만들기

In [1]:
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.6.0
1.21.4
4.11.3
1.14.0


## STEP1. STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성

 - 데이터셋은 깃허브에서 다운받거나, Huggingface datasets에서 가져올 수 있습니다. 앞에서 배운 방법들을 활용해봅시다!

In [2]:
from datasets import load_dataset

# NSMC 데이터셋 로드
dataset = load_dataset("nsmc")

# 데이터셋 정보 확인
print(dataset)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/6.33M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset nsmc downloaded and prepared to /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [3]:
train = dataset['train']
cols = train.column_names
cols

['id', 'document', 'label']

In [4]:
for i in range(5):
    for col in cols:
        print(col, ":", train[col][i])
    print('\n')

id : 9976970
document : 아 더빙.. 진짜 짜증나네요 목소리
label : 0


id : 3819312
document : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
label : 1


id : 10265843
document : 너무재밓었다그래서보는것을추천한다
label : 0


id : 9045019
document : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
label : 0


id : 6483659
document : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
label : 1




## STEP 2. klue/bert-base model 및 tokenizer 불러오기

https://huggingface.co/klue/bert-base

In [5]:
from transformers import AutoTokenizer, BertForSequenceClassification

# klue/bert-base 모델과 토크나이저 로드
model_name = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

오류 발생 ==> pytorch 기반 모델이다 보니 tensorflow로 변환해야 됨.

In [6]:
from transformers import TFBertForSequenceClassification

# PyTorch 모델을 TensorFlow로 변환
model_tf = TFBertForSequenceClassification.from_pretrained(model_name, from_pt=True, num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## STEP 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기

In [7]:
# 고정 길이 패딩 = 128
def preprocess_function(examples):
    return tokenizer(examples["document"], truncation=True, padding="max_length", max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [8]:
# # 동적 패딩
# def preprocess_function_dynamic(examples):
#     return tokenizer(examples["document"], truncation=True, padding=True)

# encoded_dataset_dynamic = dataset.map(preprocess_function_dynamic, batched=True)
# encoded_dataset_dynamic.set_format(type="tensorflow", columns=["input_ids", "attention_mask", "label"])

In [9]:
# 80% train / 20% validation 비율로 분리
split_dataset = encoded_dataset["train"].train_test_split(test_size=0.2, seed=42)

# 분리된 데이터 확인
train_data = split_dataset["train"]
valid_data = split_dataset["test"]
test_data = encoded_dataset["test"]

print(train_data)
print(valid_data)
print(test_data)

Dataset({
    features: ['attention_mask', 'document', 'id', 'input_ids', 'label', 'token_type_ids'],
    num_rows: 120000
})
Dataset({
    features: ['attention_mask', 'document', 'id', 'input_ids', 'label', 'token_type_ids'],
    num_rows: 30000
})
Dataset({
    features: ['attention_mask', 'document', 'id', 'input_ids', 'label', 'token_type_ids'],
    num_rows: 50000
})


## STEP 4. Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기
 - 데이터 전처리, TrainingArguments 등을 조정하여 모델의 정확도를 90% 이상으로 끌어올려봅시다.

In [ ]:
from transformers import Trainer, TrainingArguments
import numpy as np
from datasets import load_metric

# 정확도 측정 함수 정의
metric = load_metric('glue', 'mrpc')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs",
    logging_steps=100,
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=valid_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 모델 학습 시작
trainer.train()

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 120000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 22500


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.252400,0.251635,0.898267,0.899433


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 30000
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-7500
Configuration saved in ./results/checkpoint-7500/config.json
Model weights saved in ./results/checkpoint-7500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-7500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-7500/special_tokens_map.json


In [ ]:
trainer.evaluate(test_data)

## STEP 5. Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교
 - 아래 링크를 바탕으로 bucketing과 dynamic padding이 무엇인지 알아보고, 이들을 적용하여 model을 학습시킵니다.

Data Collator https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/data_collator  
Trainer.TrainingArguments 의 group_by_length https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments

- STEP 4에 학습한 결과와 bucketing을 적용하여 학습시킨 결과를 비교해보고, 모델 성능 향상과 훈련 시간 두 가지 측면에서 각각 어떤 이점이 있는지 비교해봅시다.